In [2]:
import sys
import os
sys.path.append(os.path.abspath('..'))

In [81]:
import os
import numpy as np
import pandas as pd
import swifter
from sklearn.model_selection import train_test_split
import utils
import itertools
from scipy.sparse import lil_matrix, csr_matrix, hstack
import scipy
import pickle
from operator import is_not
from functools import partial

from tqdm import tqdm_notebook as tqdm

pd.options.display.max_rows = 80
pd.options.display.max_columns = 80

In [5]:
DATA_PATH = "/work/IDAO/muon-detection/data/"

# load full train
train_df = utils.load_train_hdf(DATA_PATH)

In [6]:
train_df.shape

(5445705, 79)

In [7]:
train_df.dtypes

ncl[0]                 uint16
ncl[1]                 uint16
ncl[2]                 uint16
ncl[3]                 uint16
avg_cs[0]             float32
avg_cs[1]             float32
avg_cs[2]             float32
avg_cs[3]             float32
ndof                    uint8
MatchedHit_TYPE[0]       int8
MatchedHit_TYPE[1]       int8
MatchedHit_TYPE[2]       int8
MatchedHit_TYPE[3]       int8
MatchedHit_X[0]       float32
MatchedHit_X[1]       float32
MatchedHit_X[2]       float32
MatchedHit_X[3]       float32
MatchedHit_Y[0]       float32
MatchedHit_Y[1]       float32
MatchedHit_Y[2]       float32
MatchedHit_Y[3]       float32
MatchedHit_Z[0]       float32
MatchedHit_Z[1]       float32
MatchedHit_Z[2]       float32
MatchedHit_Z[3]       float32
MatchedHit_DX[0]      float32
MatchedHit_DX[1]      float32
MatchedHit_DX[2]      float32
MatchedHit_DX[3]      float32
MatchedHit_DY[0]      float32
MatchedHit_DY[1]      float32
MatchedHit_DY[2]      float32
MatchedHit_DY[3]      float32
MatchedHit

In [8]:
train_df.head()

,ncl[0],ncl[1],ncl[2],ncl[3],avg_cs[0],avg_cs[1],avg_cs[2],avg_cs[3],ndof,MatchedHit_TYPE[0],MatchedHit_TYPE[1],MatchedHit_TYPE[2],MatchedHit_TYPE[3],MatchedHit_X[0],MatchedHit_X[1],MatchedHit_X[2],MatchedHit_X[3],MatchedHit_Y[0],MatchedHit_Y[1],MatchedHit_Y[2],MatchedHit_Y[3],MatchedHit_Z[0],MatchedHit_Z[1],MatchedHit_Z[2],MatchedHit_Z[3],MatchedHit_DX[0],MatchedHit_DX[1],MatchedHit_DX[2],MatchedHit_DX[3],MatchedHit_DY[0],MatchedHit_DY[1],MatchedHit_DY[2],MatchedHit_DY[3],MatchedHit_DZ[0],MatchedHit_DZ[1],MatchedHit_DZ[2],MatchedHit_DZ[3],MatchedHit_T[0],MatchedHit_T[1],MatchedHit_T[2],MatchedHit_T[3],MatchedHit_DT[0],MatchedHit_DT[1],MatchedHit_DT[2],MatchedHit_DT[3],Lextra_X[0],Lextra_X[1],Lextra_X[2],Lextra_X[3],Lextra_Y[0],Lextra_Y[1],Lextra_Y[2],Lextra_Y[3],NShared,Mextra_DX2[0],Mextra_DX2[1],Mextra_DX2[2],Mextra_DX2[3],Mextra_DY2[0],Mextra_DY2[1],Mextra_DY2[2],Mextra_DY2[3],FOI_hits_N,FOI_hits_X,FOI_hits_Y,FOI_hits_Z,FOI_hits_DX,FOI_hits_DY,FOI_hits_DZ,FOI_hits_T,FOI_hits_DT,FOI_hits_S,PT,P,sWeight,particle_type,label,kinWeight,weight
0,47,31,13,15,2.000000,1.580645,1.153846,1.133333,8,2,2,2,2,-2685.080078,-2954.300049,-3140.969971,-3609.169922,1373.348145,1481.262207,1590.397461,1701.067627,15123.044922,16327.632812,17520.826172,18729.324219,25.500000,27.50,118.0,126.0,126.278549,136.278488,146.278412,156.278351,33.954948,33.990959,34.026966,34.062977,3,2,7,5,0,0,0,0,-2665.387939,-2917.874023,-3170.360107,-3422.846436,1394.170532,1503.675171,1613.179688,1722.684326,0,345.493927,986.100830,2201.950195,4295.085449,333.436310,951.685425,2125.099854,4145.181641,6,"[-2685.08, -2583.08, -2954.3, -3376.97, -3140....","[1373.3481, 1373.3481, 1481.2622, 1590.3975, 1...","[15123.045, 15123.045, 16327.633, 17520.826, 1...","[25.5, 25.5, 27.5, 118.0, 118.0, 126.0]","[126.27855, 126.27855, 136.27849, 146.27841, 1...","[33.95495, 33.95495, 33.99096, 34.026966, 34.0...","[3, 11, 2, 7, 7, 5]","[0, 8, 0, 0, 0, 0]","[0, 0, 1, 2, 2, 3]",1764.925899,12646.817591,1.199231,1,1,1.85646,2.226325
1,92,19,11,26,2.750000,2.789474,1.363636,1.230769,8,2,2,2,1,739.549988,852.510010,827.000000,758.719971,2387.722168,2575.974609,2764.196533,2952.103271,15212.597656,16420.576172,17614.853516,18824.429688,25.500000,27.50,118.0,756.0,126.278549,136.278488,146.278412,156.278351,33.954948,33.990959,34.026966,34.062977,9,10,8,4,1,-1,0,4,765.309204,844.153198,922.997192,1001.841187,2420.164307,2610.679199,2801.193848,2991.708740,1,62.960335,181.760788,405.539215,777.429138,64.267189,185.533554,413.956909,793.566040,4,"[739.55, 852.51, 827.0, 758.72]","[2387.7222, 2575.9746, 2764.1965, 2952.1033]","[15212.598, 16420.576, 17614.854, 18824.43]","[25.5, 27.5, 118.0, 756.0]","[126.27855, 136.27849, 146.27841, 156.27835]","[33.95495, 33.99096, 34.026966, 34.062977]","[9, 10, 8, 4]","[1, -1, 0, 4]","[0, 1, 2, 3]",4511.281266,28556.630027,1.145788,1,1,1.85646,2.127109
2,100,21,11,12,2.930000,2.428571,1.181818,1.083333,8,2,2,2,2,-1595.130005,-1748.449951,-1933.969971,-2066.669922,-318.644745,-344.529694,-221.993256,-235.322327,15317.953125,16522.060547,17598.300781,18806.351562,12.750000,13.75,59.0,63.0,63.078957,68.078926,73.078896,78.078857,33.727364,33.745369,33.763374,33.781380,10,3,3,5,0,0,0,0,-1590.582642,-1745.545898,-1900.509277,-2055.472656,-282.487946,-304.673065,-326.858185,-349.043304,0,161.833542,447.806335,996.013245,1931.484009,159.182327,440.469940,979.695190,1899.839233,9,"[-1595.13, -681.58, -1775.95, -1761.95, -1748....","[-318.64474, -222.24156, -344.5297, -207.12927...","[15317.953, 15117.299, 16522.06, 16405.555, 16...","[12.75, 76.5, 13.75, 13.75, 13.75, 13.75, 20.7...","[63.078957, 31.479162, 68.078926, 68.078926, 6...","[33.727364, 33.613575, 33.74537, 33.74537, 33....","[10, 4, 13, 13, 3, 3, 15, 3, 5]","[0, 4, 10, 6, 0, -4, 15, 0, 0]","[0, 0, 1, 1, 1, 1, 1, 2, 3]",1062.550086,17491.702634,1.139144,1,1,3.77936,4.305235
3,74,27,13,20,3.067568,2.518518,1.923077,1.300000,8,2,2,2,2,-1708.829102,-1816.949951,-1933.969971,-2066.669922,-191.404434,

In [38]:
points_for_station = [[], [], [], []]

for row in tqdm(train_df.itertuples()):
    mask = np.array(row.FOI_hits_S)
    X = np.array(row.FOI_hits_X)
    Y = np.array(row.FOI_hits_Y)
    Z = np.array(row.FOI_hits_Z)
    for i in range(4):
        points_for_station[i].extend(list(zip(X[mask==i], Y[mask==i], Z[mask==i])))

In [39]:
int_points_for_station = [[], [], [], []]

for row in tqdm(train_df.itertuples()):
    mask = np.array(row.FOI_hits_S)
    X = np.array(row.FOI_hits_X).astype(int)
    Y = np.array(row.FOI_hits_Y).astype(int)
    Z = np.array(row.FOI_hits_Z).astype(int)
    for i in range(4):
        int_points_for_station[i].extend(list(zip(X[mask==i], Y[mask==i], Z[mask==i])))

In [41]:
len(set.union(*[set(points_for_station[i]) for i in range(4)]))

108641

In [40]:
len(set.union(*[set(int_points_for_station[i]) for i in range(4)]))

108641

In [49]:
ONE_HOT_FEATURES = 108641

Здесь и далее будем всегда использовать FOI_hits скастованные к int

In [42]:
all_points_set = set.union(*[set(int_points_for_station[i]) for i in range(4)])

In [43]:
all_points_sorted = sorted(all_points_set)

In [44]:
all_points_sorted[:10]

[(-5910, -4852, 18906),
 (-5910, -4226, 18908),
 (-5910, -3600, 18911),
 (-5910, -2974, 18913),
 (-5910, -2348, 18915),
 (-5910, -1722, 18917),
 (-5910, -1095, 18920),
 (-5910, -469, 18922),
 (-5910, 157, 18924),
 (-5910, 782, 18927)]

In [45]:
point_to_one_hot_idx = {point: idx for idx, point in enumerate(all_points_sorted)}

In [46]:
point_to_one_hot_idx[(-5910, -4852, 18906)]

0

In [58]:
train_foi_hits_one_hot = lil_matrix((len(train_df), ONE_HOT_FEATURES), dtype=np.int8)

for row in tqdm(train_df.itertuples(), total=len(train_df)):
    X = np.array(row.FOI_hits_X).astype(int)
    Y = np.array(row.FOI_hits_Y).astype(int)
    Z = np.array(row.FOI_hits_Z).astype(int)
    
    train_foi_hits_one_hot[row.Index, list(map(lambda point: point_to_one_hot_idx[point], zip(X, Y, Z)))] = 1

In [59]:
train_foi_hits_one_hot = train_foi_hits_one_hot.tocsr()

In [60]:
train_foi_hits_one_hot.shape

(5445705, 108641)

In [61]:
train_foi_hits_one_hot.data.nbytes + train_foi_hits_one_hot.indptr.nbytes + train_foi_hits_one_hot.indices.nbytes

226008704

In [63]:
train_simple_features_csr = csr_matrix(train_df[utils.SIMPLE_FEATURE_COLUMNS].values)

In [64]:
train_simple_features_csr.shape

(5445705, 65)

In [67]:
train_simple_features_csr.dtype

dtype('float64')

In [68]:
train_all_features_csr = hstack([train_simple_features_csr, train_foi_hits_one_hot])

In [70]:
train_all_features_csr.shape

(5445705, 108706)

In [72]:
scipy.sparse.save_npz('../data/train_all_features_csr.npz', train_all_features_csr)

### Repeat preprocessing for test data

In [73]:
test_df = pd.read_hdf('../data/test_public_v2.hdf')

In [82]:
test_foi_hits_one_hot = lil_matrix((len(test_df), ONE_HOT_FEATURES), dtype=np.int8)

for row in tqdm(test_df.itertuples(), total=len(test_df)):
    X = np.array(row.FOI_hits_X).astype(int)
    Y = np.array(row.FOI_hits_Y).astype(int)
    Z = np.array(row.FOI_hits_Z).astype(int)
    
    test_foi_hits_one_hot[row.Index, list(filter(partial(is_not, None), map(lambda point: point_to_one_hot_idx.get(point, None), zip(X, Y, Z))))] = 1

test_foi_hits_one_hot = test_foi_hits_one_hot.tocsr()

In [83]:
test_simple_features_csr = csr_matrix(test_df[utils.SIMPLE_FEATURE_COLUMNS].values)

In [84]:
test_all_features_csr = hstack([test_simple_features_csr, test_foi_hits_one_hot])

In [85]:
test_all_features_csr.shape

(726095, 108706)

In [86]:
scipy.sparse.save_npz('../data/test_all_features_csr.npz', test_all_features_csr)